In [1]:
import pandas as pd

from selenium import webdriver

import warnings
warnings.filterwarnings('ignore')

from selenium.webdriver.common.by import By

from selenium.webdriver.chrome.options import Options

import concurrent.futures 

from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, TimeoutException

from bs4 import BeautifulSoup as bs
import requests as req

import time

In [2]:
#Cookies 
options=Options()
options.add_argument(r'user-data-dir=C:\Users\matia\anaconda3\Lib\site-packages\selenium\cookies') #Cargamos las cookies

In [3]:
url = 'https://www.transfermarkt.es/schnellsuche/ergebnis/schnellsuche?query=abner'

PATH = 'driver/chromedriver.exe'

In [4]:
driver = webdriver.Chrome(PATH, options=options)

driver.get(url)

In [5]:
#Scrapeo para sacar el valor de un jugador
driver.find_elements(By.CSS_SELECTOR, '#yw0 > table > tbody > tr:nth-child(1) > td.rechts.hauptlink')[0].text

'7,00 mill. €'

In [6]:
#Hago una funcion para scrapear el valor de todos mis jugadores
def valor(futbolista,PATH):
    
    driver = webdriver.Chrome(PATH) 
    time.sleep(2)
    driver.get(f'https://www.transfermarkt.es/schnellsuche/ergebnis/schnellsuche?query={futbolista}')
    time.sleep(2)
    try:
        valor = driver.find_elements(By.CSS_SELECTOR, '#yw0 > table > tbody > tr:nth-child(1) > td.rechts.hauptlink')[0].text
    except:
        valor = '0'
        print(f'No se encontro el valor de {futbolista}')

    driver.quit()
    return valor


In [7]:
#Pruebo la funcion
valor('Carvajal',PATH)

'12,00 mill. €'

# Se lo aplico a mi tabla completa

In [8]:
stats2  = pd.read_csv('../../csv/csv_limpios/stats.csv')

In [9]:
lista_jugadores = stats2['Jugador'].tolist()

In [10]:
with concurrent.futures.ThreadPoolExecutor(max_workers=7) as executor:  # Limita el número de hilos  
    futures = [executor.submit(valor, jugador, PATH) for jugador in lista_jugadores]
    resultados = [f.result() for f in concurrent.futures.as_completed(futures)]

No se encontro el valor de Paulo Oliveira
No se encontro el valor de Djené
No se encontro el valor de Rubén Peña
No se encontro el valor de Dani Castellano
No se encontro el valor de Javi Castellano
No se encontro el valor de Chory Castro


TimeoutException: Message: timeout: Timed out receiving message from renderer: 300.000
  (Session info: chrome=119.0.6045.199)
Stacktrace:
	GetHandleVerifier [0x00007FF7ADD182B2+55298]
	(No symbol) [0x00007FF7ADC85E02]
	(No symbol) [0x00007FF7ADB405AB]
	(No symbol) [0x00007FF7ADB2E997]
	(No symbol) [0x00007FF7ADB2E7A8]
	(No symbol) [0x00007FF7ADB2CF0E]
	(No symbol) [0x00007FF7ADB2D9CE]
	(No symbol) [0x00007FF7ADB3A20F]
	(No symbol) [0x00007FF7ADB4B551]
	(No symbol) [0x00007FF7ADB4FC1A]
	(No symbol) [0x00007FF7ADB2E032]
	(No symbol) [0x00007FF7ADB4B42A]
	(No symbol) [0x00007FF7ADBBA9C8]
	(No symbol) [0x00007FF7ADBA1E83]
	(No symbol) [0x00007FF7ADB7670A]
	(No symbol) [0x00007FF7ADB77964]
	GetHandleVerifier [0x00007FF7AE090AAB+3694587]
	GetHandleVerifier [0x00007FF7AE0E728E+4048862]
	GetHandleVerifier [0x00007FF7AE0DF173+4015811]
	GetHandleVerifier [0x00007FF7ADDB47D6+695590]
	(No symbol) [0x00007FF7ADC90CE8]
	(No symbol) [0x00007FF7ADC8CF34]
	(No symbol) [0x00007FF7ADC8D062]
	(No symbol)

In [11]:
stats2['Valor Mercado'] = resultados

NameError: name 'resultados' is not defined

# Lo aplico a temporada 2022


In [ ]:
df = pd.read_csv('../../csv/csv_limpios/jugadores_2022_lp.csv')

In [ ]:
lista_jugadores_22 = df['Jugador'].tolist()

In [ ]:
with concurrent.futures.ThreadPoolExecutor(max_workers=10) as executor:  # Limita el número de hilos  
    futures = [executor.submit(valor, jugador, PATH) for jugador in lista_jugadores_22]
    resultados = [f.result() for f in concurrent.futures.as_completed(futures)]

In [ ]:
df['Valor Mercado'] = resultados

# Exporto a csv limpio

In [ ]:
df2.to_csv('../../csv/csv_limpios/jugadores_2023_lp.csv', index=False)